In [1]:
import datetime as dt
import pandas as pd
import os
import xml.etree.ElementTree as ET

In [2]:
!pip3 install astral

In [3]:
%cd /content/drive/MyDrive/Colab_Weather_API/lib
import new_weather as nw
import weather_dict as wd
from importlib import reload

/content/drive/MyDrive/Colab_Weather_API/lib


In [4]:
%cd /content/drive/MyDrive/Colab_Weather_API/lib
reload(nw)
reload(wd)

/content/drive/MyDrive/Colab_Weather_API/lib


<module 'weather_dict' from '/content/drive/MyDrive/Colab_Weather_API/lib/weather_dict.py'>

# serviceKey 생성

In [5]:
lib_path = "/content/drive/MyDrive/Colab_PRIVATE/weather_api"
service_key = nw.get_service_key(lib_path)

In [6]:
# API
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
import xml.etree.ElementTree as ET

# API 호출

In [7]:
from datetime import datetime as dt

### 초단기실황, 초단기예보, 단기예보, 예보버전 정보를 조회하는 서비스입니다. 

* **초단기실황정보**는 예보 구역에 대한 대표 AWS 관측값을, 
* **초단기예보**는 예보시점부터 6시간 이내의 예보를, 
* **단기예보**는 예보기간과 구역을 시공간적으로 세분화한 예보를 제공합니다.

In [8]:
# sample URL
today_date = dt.now().strftime('%Y%m%d')
service_key = service_key

# 초단기실황조회
url_getUltraSrtNcst = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?serviceKey={service_key}&numOfRows=10&pageNo=1&base_date={today_date}&base_time=0600&nx=55&ny=127'
# 초단기예보조회
url_getUltraSrtFcst = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst?serviceKey={service_key}&numOfRows=10&pageNo=1&base_date={today_date}&base_time=0630&nx=55&ny=127'
# 단기예보조회
url_getVilageFcst = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey={service_key}&numOfRows=10&pageNo=1&base_date={today_date}&base_time=0500&nx=55&ny=127'
# 예보버전조회
url_getFcstVersion = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getFcstVersion?serviceKey={service_key}&numOfRows=10&pageNo=1&ftype=ODAM&basedatetime={today_date}0800'

In [9]:
# url_getFcstVersion

## 초단기실황조회

In [10]:
def getUltraSrtNcst(service_key:str, arg_dict:dict):
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
    ServiceKey = service_key
    queryParams = '?' + urlencode({ 
                                   quote_plus('ServiceKey') : ServiceKey, 
                                   quote_plus('pageNo') : arg_dict['pageNo'], 
                                   quote_plus('numOfRows') : str(arg_dict['numOfRows']), 
                                   quote_plus('base_date') : arg_dict['base_date'], 
                                   quote_plus('base_time') : arg_dict['base_time'], 
                                   quote_plus('nx') : arg_dict['nx'], 
                                   quote_plus('ny') : arg_dict['ny'], 
                               
                                  })
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    response_body = urlopen(request).read()
    return response_body

In [11]:
arg_dict = \
{
    'numOfRows':'100',
    'pageNo':'1',
    'base_date':today_date,
    'base_time':'0630',
    'nx':'55',
    'ny':'127'
 }

In [12]:
response_body = getUltraSrtNcst(service_key, arg_dict)
xtree = ET.fromstring(response_body)
nw.checker(xtree)

In [13]:
keys = ('baseDate', 'baseTime', 'category', 'nx', 'ny', 'obsrValue')

In [14]:
items = xtree.find('body').find('items').findall('item')
master = []
for item in items:
    sub_dict = {}
    for key in keys:
        sub_dict[key] = item.find(key).text
    master.append(sub_dict)

In [15]:
mdf = pd.DataFrame(master)
mdf.sort_values(by=['category'])

,baseDate,baseTime,category,nx,ny,obsrValue
0,20210724,0600,PTY,55,127,0
1,20210724,0600,REH,55,127,83
2,20210724,0600,RN1,55,127,0
3,20210724,0600,T1H,55,127,28.2
4,20210724,0600,UUU,55,127,-0.3
5,20210724,0600,VEC,55,127,62
6,20210724,0600,VVV,55,127,-0.1
7,20210724,0600,WSD,55,127,0.4


In [16]:
mdf.category.value_counts()

VEC    1
T1H    1
PTY    1
UUU    1
VVV    1
RN1    1
WSD    1
REH    1
Name: category, dtype: int64

## 초단기예보조회

In [17]:
def getUltraSrtFcst(service_key:str, arg_dict:dict):
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst'
    ServiceKey = service_key
    queryParams = '?' + urlencode({ 
                                   quote_plus('ServiceKey') : ServiceKey, 
                                   quote_plus('pageNo') : arg_dict['pageNo'], 
                                   quote_plus('numOfRows') : str(arg_dict['numOfRows']), 
                                   quote_plus('base_date') : arg_dict['base_date'], 
                                   quote_plus('base_time') : arg_dict['base_time'], 
                                   quote_plus('nx') : arg_dict['nx'], 
                                   quote_plus('ny') : arg_dict['ny'], 
                               
                                  })
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    response_body = urlopen(request).read()
    return response_body

In [18]:
arg_dict = \
{
    'numOfRows':'100',
    'pageNo':'1',
    'base_date':today_date,
    'base_time':'1130',
    'nx':'55',
    'ny':'127'
 }

In [19]:
response_body = getUltraSrtFcst(service_key, arg_dict)
xtree = ET.fromstring(response_body)
nw.checker(xtree)

In [20]:
keys = ('baseDate', 'baseTime', 'category', 'fcstDate', 'fcstTime', 'fcstValue', 'nx', 'ny')
# keys = ('category', 'fcstTime', 'fcstValue')

In [21]:
items = xtree.find('body').find('items').findall('item')
master = []
for item in items:
    sub_dict = {}
    for key in keys:
        sub_dict[key] = item.find(key).text
    master.append(sub_dict)

In [22]:
mdf = pd.DataFrame(master)
print(mdf.shape)
mdf.head()

(60, 8)


,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20210724,1130,LGT,20210724,1200,0,55,127
1,20210724,1130,LGT,20210724,1300,0,55,127
2,20210724,1130,LGT,20210724,1400,0,55,127
3,20210724,1130,LGT,20210724,1500,0,55,127
4,20210724,1130,LGT,20210724,1600,0,55,127


## 단기예보조회

In [23]:
def getVilageFcst(service_key:str, arg_dict:dict):
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'
    ServiceKey = service_key
    queryParams = '?' + urlencode({ 
                                   quote_plus('ServiceKey') : ServiceKey, 
                                   quote_plus('pageNo') : arg_dict['pageNo'], 
                                   quote_plus('numOfRows') : str(arg_dict['numOfRows']), 
                                   quote_plus('base_date') : arg_dict['base_date'], 
                                   quote_plus('base_time') : arg_dict['base_time'], 
                                   quote_plus('nx') : arg_dict['nx'], 
                                   quote_plus('ny') : arg_dict['ny'], 
                                  })
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    response_body = urlopen(request).read()
    return response_body

In [24]:
arg_dict = \
{
    'numOfRows':'1000',
    'pageNo':'1',
    'base_date':today_date,
    'base_time':'0500',
    'nx':'55',
    'ny':'127'
 }

In [25]:
response_body = getVilageFcst(service_key, arg_dict)
xtree = ET.fromstring(response_body)
nw.checker(xtree)

In [26]:
keys = ('baseDate', 'baseTime', 'category', 'fcstDate', 'fcstTime', 'fcstValue', 'nx', 'ny')
# keys = ('category', 'fcstTime', 'fcstValue')

In [27]:
items = xtree.find('body').find('items').findall('item')
master = []
for item in items:
    sub_dict = {}
    for key in keys:
        sub_dict[key] = item.find(key).text
    master.append(sub_dict)

In [28]:
mdf = pd.DataFrame(master)
"""TODO
'TMX'(일 최고기온), 'TMN'(일 최저기온) 값을 보간 또는 드롭 할 수 있게 선택
"""
mdf.drop(mdf.index[mdf.category == 'TMX'], inplace=True)
mdf.drop(mdf.index[mdf.category == 'TMN'], inplace=True)
print(mdf.shape)
mdf.head()

(737, 8)


,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20210724,0500,TMP,20210724,0600,26,55,127
1,20210724,0500,UUU,20210724,0600,-0.7,55,127
2,20210724,0500,VVV,20210724,0600,1.2,55,127
3,20210724,0500,VEC,20210724,0600,146,55,127
4,20210724,0500,WSD,20210724,0600,1.4,55,127


## Rehsape 데이터프레임

In [29]:
sorted_df = nw.sort_fcst(mdf)
df = nw.reshape_sdf(sorted_df)
org_col = df.columns
df.shape

(67, 11)

##한국어 컬럼명

In [30]:
# fcstValue Dictionary
"""CHECK
단기예보조회는 vilagefcst_dict를 사용하지 않고 ultrasrtncst_dict를 사용한다.
"""
kor_col = [wd.ultrasrtncst_dict[col]['항목명'] for col in df.columns]
# kor_categories = [wd.ultrasrtfcst_dict[col]['항목명'] for col in categories]

In [31]:
df.columns = kor_col
df.head()

,1시간 강수량,강수확률,강수형태,습도,하늘상태,1시간 신적설,1시간 기온,풍속(동서성분),풍향,풍속(남북성분),풍속
202107240600,0,0,0,85,1,0,26,-0.7,146,1.2,1.4
202107240700,0,0,0,80,1,0,27,-0.7,158,2,2.2
202107240800,0,0,0,70,1,0,28,-0.7,160,2.2,2.3
202107240900,0,0,0,65,1,0,30,-0.5,164,2.1,2.2
202107241000,0,20,0,60,3,0,31,-0.7,148,1.3,1.5
